In [2]:
from tqdm.notebook import tqdm
from setup import setup
setup()

In [3]:
import datasets

from torchvision import transforms

imagenet = datasets.load_dataset("ILSVRC/imagenet-1k", trust_remote_code=True)
print(imagenet)

/opt/conda/envs/lightning/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 1281167
    })
    validation: Dataset({
        features: ['image', 'label'],
        num_rows: 50000
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 100000
    })
})


In [4]:
from src.supcl.util import TwoCropTransform
from torchvision import transforms

mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)
normalize = transforms.Normalize(mean=mean, std=std)

train_transform = transforms.Compose(
    [
        autoaugment.AutoAugment(policy=autoaugment.AutoAugmentPolicy.IMAGENET),
        transforms.ToTensor(),
        normalize,
    ]
)

In [5]:
from src.supcl.util import load_imagenet_hf
imagenet_train = load_imagenet_hf(object(), TwoCropTransform(train_transform))['train']
imagenet_train[0]

{'image': [tensor([[[1.3584, 1.3584, 1.3584,  ..., 1.0331, 1.0331, 0.9988],
           [1.4783, 1.4783, 1.4440,  ..., 1.1015, 1.1529, 1.1529],
           [1.6153, 1.6153, 1.6324,  ..., 0.5364, 1.1529, 1.3070],
           ...,
           [1.4440, 1.4269, 1.4783,  ..., 1.5468, 1.5468, 1.5297],
           [1.4098, 1.4269, 1.4269,  ..., 1.5297, 1.5468, 1.5468],
           [1.4269, 1.4269, 1.4269,  ..., 1.5468, 1.5468, 1.5468]],
  
          [[1.5357, 1.5532, 1.5357,  ..., 1.2031, 1.1856, 1.1856],
           [1.6758, 1.6758, 1.6583,  ..., 1.2556, 1.3431, 1.3431],
           [1.8158, 1.8158, 1.7983,  ..., 0.5028, 1.3081, 1.5182],
           ...,
           [1.6057, 1.6408, 1.6583,  ..., 1.7108, 1.7108, 1.6933],
           [1.6057, 1.6408, 1.6583,  ..., 1.6933, 1.7108, 1.7108],
           [1.5882, 1.6057, 1.6408,  ..., 1.7108, 1.7108, 1.7108]],
  
          [[1.7860, 1.7860, 1.7860,  ..., 1.3851, 1.3677, 1.3677],
           [1.9254, 1.9254, 1.8905,  ..., 1.4025, 1.5071, 1.5071],
           [2

In [6]:
import matplotlib.pyplot as plt
import torch
import torchvision.transforms.autoaugment as autoaugment

# Get first image from imagenet_train
first_sample = imagenet_train[0]
image = first_sample['image'][0] # Get first view of the two-crop transform

# Create AutoAugment transform for ImageNet
auto_augment = autoaugment.AutoAugment(policy=autoaugment.AutoAugmentPolicy.IMAGENET)

# Apply AutoAugment 3 times to get different random augmentations
aug1 = auto_augment(image)
aug2 = auto_augment(image) 
aug3 = auto_augment(image)

# Plot original and augmented images
fig, axes = plt.subplots(2, 2, figsize=(10, 10))

# Helper function to denormalize and convert tensor to numpy for plotting
def denormalize(tensor):
    # Denormalize using ImageNet mean and std
    denorm = transforms.Normalize(
        mean=[-m/s for m, s in zip(mean, std)],
        std=[1/s for s in std]
    )
    return denorm(tensor).permute(1, 2, 0).clip(0, 1).numpy()

# Plot original and augmented images
axes[0,0].imshow(denormalize(image))
axes[0,0].set_title('Original')
axes[0,1].imshow(denormalize(aug1))
axes[0,1].set_title('AutoAugment 1')
axes[1,0].imshow(denormalize(aug2))
axes[1,0].set_title('AutoAugment 2') 
axes[1,1].imshow(denormalize(aug3))
axes[1,1].set_title('AutoAugment 3')

for ax in axes.flat:
    ax.axis('off')

plt.tight_layout()
plt.show()


TypeError: Threshold should be less than bound of img.

In [2]:
dataset['train']

Dataset({
    features: ['image', 'label'],
    num_rows: 1281167
})

In [10]:
from torchvision import datasets as torchvision_datasets
from src.supcl.util import TwoCropTransform
cifar10_dataset = torchvision_datasets.CIFAR10(
    root='../datasets/', transform=TwoCropTransform(train_transform), download=True
)

Files already downloaded and verified


In [20]:
cifar10_dataset[0]

([tensor([[[-1.0219, -1.1589, -1.1418,  ..., -0.1143, -0.1143, -0.0972],
           [-1.3987, -1.5699, -1.4672,  ..., -0.3712, -0.3883, -0.4568],
           [-1.3815, -1.5014, -1.2959,  ..., -0.3198, -0.4226, -0.5253],
           ...,
           [ 0.3652,  0.0912, -0.0801,  ..., -0.2513,  0.2624,  0.2453],
           [ 0.3823,  0.1426, -0.0116,  ..., -0.5253,  0.0741, -0.0287],
           [ 0.2967,  0.1254,  0.0741,  ..., -1.0904, -0.3369, -0.1486]],
  
          [[-0.9153, -1.0553, -1.0378,  ...,  0.0126,  0.0126,  0.0301],
           [-1.3004, -1.4755, -1.3704,  ..., -0.2500, -0.2675, -0.3375],
           [-1.2829, -1.4055, -1.1954,  ..., -0.1975, -0.3025, -0.4076],
           ...,
           [ 0.5028,  0.2227,  0.0476,  ..., -0.1275,  0.3978,  0.3803],
           [ 0.5203,  0.2752,  0.1176,  ..., -0.4076,  0.2052,  0.1001],
           [ 0.4328,  0.2577,  0.2052,  ..., -0.9853, -0.2150, -0.0224]],
  
          [[-0.6890, -0.8284, -0.8110,  ...,  0.2348,  0.2348,  0.2522],
           